In [3]:
pwd

'f:\\iNeuron\\Projects\\scania_failures_2'

In [50]:
import os

import mlflow.pyfunc

from src.config.configuration_manager import ConfigurationManager
from src.components.stage_3_data_split import data_splitting_component
from src.components.stage_4_final_preprocessing import stage_4_final_processing_component
from src.utils import eval_metrics,save_yaml

os.chdir('f:\\iNeuron\\Projects\\scania_failures_2')

In [47]:
obj = ConfigurationManager()
stage_1_obj = obj.get_stage1_processing_config()
stage_2_obj = obj.get_stage2_processing_config()
data_split_obj = obj.get_data_split_config()
preprocessor_obj = obj.get_preprocessor_config()
model_obj = obj.get_model_config()

data_split_class_obj = data_splitting_component(data_split_conf = data_split_obj,
                                                stage1_processor_conf = stage_1_obj)
stage_4_final_processing_class_obj = stage_4_final_processing_component(data_split_conf = data_split_obj,
                                                                        stage_2_processor_conf = stage_2_obj,
                                                                        preprocessor_conf = preprocessor_obj)

[2024-01-26 13:14:32,527: INFO: utils: config.yaml yaml_file is loaded]
[2024-01-26 13:14:32,538: INFO: utils: params.yaml yaml_file is loaded]
[2024-01-26 13:14:32,544: INFO: utils: schema.yaml yaml_file is loaded]


In [6]:
train_data_training_set,train_data_testing_set = data_split_class_obj.data_splitting(None)

Size:  None
Pre_train_data shape:  (45000, 171) 
Pre_test_data shape:  (15000, 171)


In [7]:
transformed_train_df, transformed_test_df = stage_4_final_processing_class_obj.final_processing(train_data_training_set,
                                                                                                train_data_testing_set)

[2024-01-26 11:58:41,485: INFO: utils: config.yaml yaml_file is loaded]
[2024-01-26 11:58:41,489: INFO: utils: params.yaml yaml_file is loaded]
[2024-01-26 11:58:41,493: INFO: utils: schema.yaml yaml_file is loaded]
[2024-01-26 11:58:41,497: INFO: utils: schema.yaml yaml_file is loaded]
[2024-01-26 11:58:41,499: INFO: utils: Stage 2 Processing Commencing]
[2024-01-26 11:58:41,500: INFO: utils: Pipeline created with KnnImputer, RobustScaler]
[2024-01-26 11:58:41,501: INFO: utils: SmoteTomek obj created]
[2024-01-26 11:58:41,533: INFO: utils: Commencing pipeline transformation]
[Pipeline] ....... (step 1 of 2) Processing Knn_imputer, total=16.4min
[Pipeline] ..... (step 2 of 2) Processing Robust_Scaler, total=   0.3s
[2024-01-26 12:15:02,498: INFO: utils: Pipeline transformation complete]
[2024-01-26 12:15:02,503: INFO: utils: Commencing SmoteTomek]
[2024-01-26 12:15:37,320: INFO: utils: SmoteTomek Complete]
[2024-01-26 12:15:37,321: INFO: utils: Returning the transformed dataframe]
[202

In [8]:
transformed_train_df.shape

(88332, 171)

In [9]:
transformed_test_df.shape

(29406, 171)

In [10]:
transformed_train_df.isna().sum().unique()

array([0], dtype=int64)

In [11]:
transformed_test_df.isna().sum().unique()

array([0], dtype=int64)

In [12]:
transformed_train_df['class'].value_counts()

class
0    44166
1    44166
Name: count, dtype: int64

In [13]:
transformed_test_df['class'].value_counts()

class
0    14703
1    14703
Name: count, dtype: int64

In [14]:
transformed_train_df.head()

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000,class
0,-0.642451,0.0,-1.184510e-01,-0.130841,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.513180,-0.473398,-0.472359,-0.341886,-0.246246,-0.028516,0.000000,0.0,0.0,0
1,-0.647499,-0.5,-9.111617e-02,-0.336449,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.516161,-0.478466,-0.477442,-0.343247,-0.247308,-0.028962,0.000000,0.0,0.0,0
2,-0.646405,0.0,-5.466970e-02,-0.205607,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.515376,-0.478164,-0.477356,-0.343320,-0.247344,-0.028962,0.000000,0.0,0.0,0
3,0.221708,0.0,4.894240e-01,1.425234,0.0,0.0,0.0,0.0,0.0,0.0,...,0.657849,0.963839,0.682807,0.064776,-0.110265,-0.007890,0.000000,0.0,0.0,0
4,-0.647204,-0.5,1.386727e+06,-0.345794,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.516105,-0.478462,-0.477416,-0.343079,-0.245686,-0.028962,0.089216,0.0,0.0,0


In [15]:
x_train, y_train = transformed_train_df.drop(columns = 'class'),transformed_train_df['class']
x_test, y_test = transformed_test_df.drop(columns = 'class'),transformed_test_df['class']

##### Load the model

In [19]:
model = mlflow.pyfunc.load_model('file:artifacts/model/challenger_hyperopt_SGD_Classifier')

In [22]:
y_pred = model.predict(x_test)
eval_metrics(y_true = y_test, 
             y_pred = y_pred)

{'Balanced_Accuracy_Score': 0.6172209753111609,
 'F1_Score': 0.6819260766361479,
 'Accuracy_Score': 0.6172209753111609,
 'Cost': 1404690.0}

In [28]:
train_data_training_set_, train_data_testing_set_ = data_split_class_obj.data_splitting(1500)
transformed_train_df_, transformed_test_df_ = stage_4_final_processing_class_obj.final_processing(train_data_training_set_,
                                                                                                train_data_testing_set_)
x_train_, y_train_ = transformed_train_df_.drop(columns = 'class'),transformed_train_df_['class']
x_test_, y_test_ = transformed_test_df_.drop(columns = 'class'),transformed_test_df_['class']

Size:  1500
Pre_train_data shape:  (1125, 171) 
Pre_test_data shape:  (375, 171)
[2024-01-26 12:38:24,015: INFO: utils: config.yaml yaml_file is loaded]
[2024-01-26 12:38:24,020: INFO: utils: params.yaml yaml_file is loaded]
[2024-01-26 12:38:24,025: INFO: utils: schema.yaml yaml_file is loaded]
[2024-01-26 12:38:24,029: INFO: utils: schema.yaml yaml_file is loaded]
[2024-01-26 12:38:24,030: INFO: utils: Stage 2 Processing Commencing]
[2024-01-26 12:38:24,031: INFO: utils: Pipeline created with KnnImputer, RobustScaler]
[2024-01-26 12:38:24,033: INFO: utils: SmoteTomek obj created]
[2024-01-26 12:38:24,035: INFO: utils: Commencing pipeline transformation]
[Pipeline] ....... (step 1 of 2) Processing Knn_imputer, total=   0.5s
[Pipeline] ..... (step 2 of 2) Processing Robust_Scaler, total=   0.0s
[2024-01-26 12:38:24,528: INFO: utils: Pipeline transformation complete]
[2024-01-26 12:38:24,530: INFO: utils: Commencing SmoteTomek]
[2024-01-26 12:38:24,590: INFO: utils: SmoteTomek Complete]

In [29]:
y_pred_ = model.predict(x_test_)
eval_metrics(y_true = y_test_, 
             y_pred = y_pred_)

{'Balanced_Accuracy_Score': 0.65,
 'F1_Score': 0.7278617710583153,
 'Accuracy_Score': 0.65,
 'Cost': 13790.0}

In [33]:
model.metadata.run_id

'3043be8b328d4809b8c693668da148ea'

In [45]:
mlflow.pyfunc.load_model(f"file:artifacts/model/hp_tuned_model/{model.metadata.artifact_path}")

mlflow.pyfunc.loaded_model:
  artifact_path: challenger_hyperopt_SGD_Classifier
  flavor: mlflow.sklearn
  run_id: 3043be8b328d4809b8c693668da148ea

In [46]:
import os

def get_most_recent_subdirectory(parent_directory):
    # Get a list of all subdirectories in the parent directory
    subdirectories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

    # If there are no subdirectories, return None
    if not subdirectories:
        return None

    # Sort the subdirectories by modification time (most recent first)
    subdirectories.sort(key=lambda d: os.path.getmtime(os.path.join(parent_directory, d)), reverse=True)

    # Return the most recent subdirectory
    return os.path.join(parent_directory, subdirectories[0])

# Example usage:
parent_directory = "artifacts/model/hp_tuned_model/"
most_recent_subdirectory = get_most_recent_subdirectory(parent_directory)

if most_recent_subdirectory:
    print(f"The most recent subdirectory is: {most_recent_subdirectory}")
else:
    print("No subdirectories found.")


The most recent subdirectory is: artifacts/model/hp_tuned_model/challenger_hyperopt_SGD_Classifier


In [53]:
model_obj.root_dir

'artifacts/model'

In [65]:
model.metadata.artifact_path

'challenger_hyperopt_SGD_Classifier'

In [64]:
from pathlib import Path


os.chdir(f"F:\iNeuron\Projects\scania_failures_2\\artifacts\model")
save_yaml(file = model.metadata.artifact_path, filepath = Path(f"{model_obj.root_dir}"))

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\model'

In [52]:
pwd

'f:\\iNeuron\\Projects\\scania_failures_2'